In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('csv') or filename.endswith('hdf'):
            print(os.path.join(dirname, filename))

             
print()

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if '__output__' not in filename:
            if filename.endswith('json'):
                print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

/kaggle/input/forest-cover-type-standard-normalized/train_noramlized.csv
/kaggle/input/forest-cover-type-standard-normalized/test_noramlized.csv
/kaggle/input/forest-cover-type-standard-normalized/train_noramlized.hdf
/kaggle/input/forest-cover-type-standard-normalized/test_noramlized.hdf
/kaggle/input/covertype-target/target.csv

/kaggle/input/featuretoos-feature-selection-framework/features_abs_diff.json
/kaggle/input/featuretoos-feature-selection-framework/features_mean_2_cols.json
/kaggle/input/featuretoos-feature-selection-framework/features_squared_root.json
/kaggle/input/featuretoos-feature-selection-framework/features_subtract_numeric.json
/kaggle/input/featuretoos-feature-selection-framework/features_multiply_numeric.json
/kaggle/input/featuretoos-feature-selection-framework/features_add_numeric.json
/kaggle/input/featuretoos-feature-selection-framework/features_mean_3_cols.json
/kaggle/input/featuretoos-feature-selection-framework/features_div_2_cols.json


In [2]:
%%time
# train_df = pd.read_csv('/kaggle/input/forest-cover-type-standard-normalized/train_noramlized.csv')
# test_df = pd.read_csv('/kaggle/input/forest-cover-type-standard-normalized/test_noramlized.csv')
# submission_df = pd.read_csv('/kaggle/input/forest-cover-type-kernels-only/sample_submission.csv')

train_df = pd.read_hdf('/kaggle/input/forest-cover-type-standard-normalized/train_noramlized.hdf')
test_df = pd.read_hdf('/kaggle/input/forest-cover-type-standard-normalized/test_noramlized.hdf')

CPU times: user 512 ms, sys: 328 ms, total: 840 ms
Wall time: 1.47 s


In [3]:
train_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,-0.367095,-0.959980,-1.597132,0.146639,-0.834074,-0.908681,0.271454,0.571653,0.281259,...,0,0,0,0,0,0,0,0,0,5
1,2,-0.381461,-0.914559,-1.715424,-0.072337,-0.932054,-0.999246,0.238732,0.703225,0.346627,...,0,0,0,0,0,0,0,0,0,5
2,3,0.130912,-0.160577,-0.887379,0.194243,0.227369,1.106379,0.696843,0.834797,-0.002005,...,0,0,0,0,0,0,0,0,0,2
3,4,0.085421,-0.015231,0.177250,0.070474,1.092853,1.038455,0.827731,0.834797,-0.285268,...,0,0,0,0,0,0,0,0,0,2
4,5,-0.369489,-1.014485,-1.715424,-0.353198,-0.850404,-0.998491,0.238732,0.659368,0.324838,...,0,0,0,0,0,0,0,0,0,5


In [4]:
test_df['Cover_Type'] = np.nan
data_df = pd.concat([train_df, test_df]).drop(columns=['Cover_Type'])

In [5]:
print(train_df.shape)
print(test_df.shape)
print(data_df.shape)

(15120, 56)
(565892, 56)
(581012, 55)


## Read Target Dataset

In [6]:
target_all = pd.read_csv('/kaggle/input/covertype-target/target.csv',
                        index_col='Id')

y_test = target_all.loc[test_df['Id']]

## Convert One-Hot encoding to label

In [7]:

# cat_soil = [col for col in data_df.columns if 'Soil' in col]
# data_df['Soil_Type'] = data_df[cat_soil].values.argmax(axis=1)+1
# data_df = data_df.drop(columns=cat_soil)


# cat_wilderness = [col for col in data_df.columns if 'Wilderness' in col]
# data_df['Wilderness_Area'] = data_df[cat_wilderness].values.argmax(axis=1)+1
# data_df = data_df.drop(columns=cat_wilderness)

In [8]:
# data_df.shape

In [9]:
# data_df.columns

In [10]:
# train_df.columns

In [11]:
import featuretools as ft
import featuretools.variable_types as vtypes
from multiprocessing import cpu_count
from IPython.display import display

In [12]:

# for i in range(0, len(ft.list_primitives()), 10):
# #     print(i)
    
#     display(ft.list_primitives()[0+i:i+10])

# Feature Engineering

### Define Transform Primitives

In [13]:
from featuretools.primitives import make_trans_primitive
from featuretools.variable_types import Numeric

def squared_root(col1, col2):
    return (col1**2 + col2**2)**0.5

Squared_Root = make_trans_primitive(function=squared_root,
                                   input_types=[Numeric, Numeric],
                                   return_type=Numeric,
                                    commutative=True, # True: combination, False: permutaion
                                   )


def abs_diff(col1, col2):
    return abs(col1 - col2)

Abs_Diff = make_trans_primitive(function=abs_diff,
                               input_types=[Numeric, Numeric],
                               return_type=Numeric,
                               commutative=True, # True: combination, False: permutaion
                               )


def mean_2_cols(col1, col2):
    return (col1.add(col2)) / 2


Mean_2_Cols = make_trans_primitive(function=mean_2_cols,
                                  input_types=[Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )


def mean_3_cols(col1, col2, col3):
    return (col1.add(col2.add(col3))) / 3

Mean_3_Cols = make_trans_primitive(function=mean_3_cols,
                                  input_types=[Numeric, Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )


def div_2_cols(col1, col2):
    eps = 10e-2
    return (col1+eps) / (col2+eps)


Div_2_Cols = make_trans_primitive(function=div_2_cols,
                                  input_types=[Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )

### Read Feature Json Files

In [14]:
from collections import OrderedDict

filename_dict = OrderedDict()

filename_dict['add_2'] ='/kaggle/input/featuretoos-feature-selection-framework/features_add_numeric.json'
filename_dict['dif_2'] = '/kaggle/input/featuretoos-feature-selection-framework/features_subtract_numeric.json'
filename_dict['abs_dif_2'] ='/kaggle/input/featuretoos-feature-selection-framework/features_abs_diff.json'
filename_dict['mul_2'] = '/kaggle/input/featuretoos-feature-selection-framework/features_multiply_numeric.json'
# filename_dict['div_2'] ='/kaggle/input/featuretoos-feature-selection-framework/features_div_2_cols.json'
filename_dict['mean_2'] ='/kaggle/input/featuretoos-feature-selection-framework/features_mean_2_cols.json'
filename_dict['mean_3'] ='/kaggle/input/featuretoos-feature-selection-framework/features_mean_3_cols.json'
filename_dict['sqrt_2'] ='/kaggle/input/featuretoos-feature-selection-framework/features_squared_root.json'

In [15]:
n_features = 0

features_list = []
features_dict = OrderedDict()
for name, filename in filename_dict.items():
    features_dict[name] = ft.load_features(filename)
    features_list = features_list + features_dict[name]
    
    print('{:10s}: num of features= {}'.format(name,
                                       len(features_dict[name])))
    
    n_features += len(features_dict[name])
    
print()
print('Total num of features=       {}'.format(n_features))

add_2     : num of features= 19
dif_2     : num of features= 21
abs_dif_2 : num of features= 21
mul_2     : num of features= 19
mean_2    : num of features= 19
mean_3    : num of features= 22
sqrt_2    : num of features= 20

Total num of features=       141


In [16]:
variable_type = {col: vtypes.Categorical for col in data_df.columns if 'Soil' in col or 'Wilderness' in col}

# variable_type['Cover_Type'] = vtypes.Categorical

print(variable_type)

{'Wilderness_Area1': <class 'featuretools.variable_types.variable.Categorical'>, 'Wilderness_Area2': <class 'featuretools.variable_types.variable.Categorical'>, 'Wilderness_Area3': <class 'featuretools.variable_types.variable.Categorical'>, 'Wilderness_Area4': <class 'featuretools.variable_types.variable.Categorical'>, 'Soil_Type1': <class 'featuretools.variable_types.variable.Categorical'>, 'Soil_Type2': <class 'featuretools.variable_types.variable.Categorical'>, 'Soil_Type3': <class 'featuretools.variable_types.variable.Categorical'>, 'Soil_Type4': <class 'featuretools.variable_types.variable.Categorical'>, 'Soil_Type5': <class 'featuretools.variable_types.variable.Categorical'>, 'Soil_Type6': <class 'featuretools.variable_types.variable.Categorical'>, 'Soil_Type7': <class 'featuretools.variable_types.variable.Categorical'>, 'Soil_Type8': <class 'featuretools.variable_types.variable.Categorical'>, 'Soil_Type9': <class 'featuretools.variable_types.variable.Categorical'>, 'Soil_Type10'

In [17]:
es = ft.EntitySet('Dataset')

es.entity_from_dataframe(entity_id='forest_table',
                         index='Id',
                        dataframe=data_df,
                         variable_types=variable_type
                        )

Entityset: Dataset
  Entities:
    forest_table [Rows: 581012, Columns: 55]
  Relationships:
    No relationships

In [18]:
es['forest_table'].variables

[<Variable: Id (dtype = index)>,
 <Variable: Elevation (dtype = numeric)>,
 <Variable: Aspect (dtype = numeric)>,
 <Variable: Slope (dtype = numeric)>,
 <Variable: Horizontal_Distance_To_Hydrology (dtype = numeric)>,
 <Variable: Vertical_Distance_To_Hydrology (dtype = numeric)>,
 <Variable: Horizontal_Distance_To_Roadways (dtype = numeric)>,
 <Variable: Hillshade_9am (dtype = numeric)>,
 <Variable: Hillshade_Noon (dtype = numeric)>,
 <Variable: Hillshade_3pm (dtype = numeric)>,
 <Variable: Horizontal_Distance_To_Fire_Points (dtype = numeric)>,
 <Variable: Wilderness_Area1 (dtype = categorical)>,
 <Variable: Wilderness_Area2 (dtype = categorical)>,
 <Variable: Wilderness_Area3 (dtype = categorical)>,
 <Variable: Wilderness_Area4 (dtype = categorical)>,
 <Variable: Soil_Type1 (dtype = categorical)>,
 <Variable: Soil_Type2 (dtype = categorical)>,
 <Variable: Soil_Type3 (dtype = categorical)>,
 <Variable: Soil_Type4 (dtype = categorical)>,
 <Variable: Soil_Type5 (dtype = categorical)>,
 <V

In [19]:
es['forest_table'].df

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
1,1,-0.367095,-0.959980,-1.597132,0.146639,-0.834074,-0.908681,0.271454,0.571653,0.281259,...,0,0,0,0,0,0,0,0,0,0
2,2,-0.381461,-0.914559,-1.715424,-0.072337,-0.932054,-0.999246,0.238732,0.703225,0.346627,...,0,0,0,0,0,0,0,0,0,0
3,3,0.130912,-0.160577,-0.887379,0.194243,0.227369,1.106379,0.696843,0.834797,-0.002005,...,0,0,0,0,0,0,0,0,0,0
4,4,0.085421,-0.015231,0.177250,0.070474,1.092853,1.038455,0.827731,0.834797,-0.285268,...,0,0,0,0,0,0,0,0,0,0
5,5,-0.369489,-1.014485,-1.715424,-0.353198,-0.850404,-0.998491,0.238732,0.659368,0.324838,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581008,581008,-0.845949,-0.033399,0.413835,-0.676902,-0.556466,-1.212072,0.893176,0.790940,-0.372426,...,0,0,0,0,0,0,0,0,0,0
581009,581009,-0.857920,-0.042483,0.295543,-0.762588,-0.638115,-1.221884,0.893176,0.790940,-0.350637,...,0,0,0,0,0,0,0,0,0,0
581010,581010,-0.869891,0.021106,0.058958,-0.795911,-0.719765,-1.225657,0.762287,0.966369,-0.110952,...,0,0,0,0,0,0,0,0,0,0
581011,581011,-0.874680,0.121031,-0.177626,-0.795911,-0.752425,-1.225657,0.565954,1.141799,0.172311,...,0,0,0,0,0,0,0,0,0,0


In [20]:
%%time
_, features_org = ft.dfs(entityset=es,
                        target_entity='forest_table')

features_list_combined = features_org + features_list

print('original features:           {}'.format(len(features_org)))
print('selected features from fils: {}'.format(len(features_list)))
print('combined features:           {}'.format(len(features_list_combined)))

print()
print('unique num of features:      {}'.format(len(set(features_list_combined))))
print()

original features:           54
selected features from fils: 141
combined features:           195

unique num of features:      195

CPU times: user 6.86 s, sys: 1.76 s, total: 8.62 s
Wall time: 7.86 s


In [21]:
%%time
features = features_list_combined
feature_matrix = ft.calculate_feature_matrix(features,
                                            entityset=es,
#                                             n_jobs=cpu_count()
                                            )

CPU times: user 11.9 s, sys: 6.79 s, total: 18.7 s
Wall time: 12.6 s


In [22]:
# %%time
# feature_matrix, features = ft.dfs(entityset=es,
#                                  target_entity='forest_table',
#                                   trans_primitives=[
# #                                                     Squared_Root,
# #                                                     Abs_Diff,
#                                                     Mean_3_Cols,
# #                                                       Div_2_Cols,
# #                                                     'add_numeric', 
# #                                                   'subtract_numeric',
# #                                                     'multiply_numeric',
# #                                                     'divide_numeric',  # divide by 0.
# #                                                     'subtract_numeric',
#                                                    ],
#                                  max_depth=1,
#                                  n_jobs=cpu_count())

In [23]:
feature_matrix = feature_matrix.reset_index()

In [24]:
feature_matrix.columns[:5]

Index(['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology'],
      dtype='object')

In [25]:
feature_matrix.shape

(581012, 196)

In [26]:
data_df.shape

(581012, 55)

In [27]:
features

[<Feature: Elevation>,
 <Feature: Aspect>,
 <Feature: Slope>,
 <Feature: Horizontal_Distance_To_Hydrology>,
 <Feature: Vertical_Distance_To_Hydrology>,
 <Feature: Horizontal_Distance_To_Roadways>,
 <Feature: Hillshade_9am>,
 <Feature: Hillshade_Noon>,
 <Feature: Hillshade_3pm>,
 <Feature: Horizontal_Distance_To_Fire_Points>,
 <Feature: Wilderness_Area1>,
 <Feature: Wilderness_Area2>,
 <Feature: Wilderness_Area3>,
 <Feature: Wilderness_Area4>,
 <Feature: Soil_Type1>,
 <Feature: Soil_Type2>,
 <Feature: Soil_Type3>,
 <Feature: Soil_Type4>,
 <Feature: Soil_Type5>,
 <Feature: Soil_Type6>,
 <Feature: Soil_Type7>,
 <Feature: Soil_Type8>,
 <Feature: Soil_Type9>,
 <Feature: Soil_Type10>,
 <Feature: Soil_Type11>,
 <Feature: Soil_Type12>,
 <Feature: Soil_Type13>,
 <Feature: Soil_Type14>,
 <Feature: Soil_Type15>,
 <Feature: Soil_Type16>,
 <Feature: Soil_Type17>,
 <Feature: Soil_Type18>,
 <Feature: Soil_Type19>,
 <Feature: Soil_Type20>,
 <Feature: Soil_Type21>,
 <Feature: Soil_Type22>,
 <Feature: S

# Prepare Train/Test Dataset

In [28]:
feature_matrix = feature_matrix.set_index('Id')

In [29]:
indice = train_df['Id']
# X_train = feature_matrix.loc[indice].drop(columns=['soil_type_table.Cover_Type'])
# y_train = feature_matrix.loc[indice, 'soil_type_table.Cover_Type'].astype('category')
# y_train = train_df['Cover_Type']
X_train = feature_matrix.loc[indice]
y_train = train_df['Cover_Type'].astype('category')

indice = test_df['Id']
# X_test = feature_matrix.loc[indice].drop(columns=['soil_type_table.Cover_Type'])
X_test = feature_matrix.loc[indice]

### Recycle `feature_matrix`

In [30]:
import gc

del feature_matrix
# del features

gc.collect()

480

In [31]:
from IPython.display import display

display(X_train.tail())
display(train_df.tail())

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,"SQUARED_ROOT(Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Hillshade_9am, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Hillshade_3pm, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Aspect, Horizontal_Distance_To_Hydrology)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Hydrology)","SQUARED_ROOT(Hillshade_9am, Horizontal_Distance_To_Hydrology)","SQUARED_ROOT(Elevation, Vertical_Distance_To_Hydrology)","SQUARED_ROOT(Aspect, Horizontal_Distance_To_Fire_Points)","SQUARED_ROOT(Aspect, Hillshade_9am)"
Id,,,,,,,,,,,,,,,,,,,,,
15116,-0.340758,0.784172,0.768711,0.146639,-0.719765,-0.795476,-1.397379,1.404943,1.719366,-0.208335,...,0.808879,1.607934,1.894466,0.797765,0.865389,0.370971,1.405052,0.796353,0.811375,1.602372
15117,-0.350335,-0.324091,0.295543,1.931773,2.350255,-0.827173,1.187676,0.089223,-0.960742,-0.169240,...,2.101419,1.447339,1.267770,1.958770,0.898304,1.963283,2.267668,2.376223,0.365619,1.231101
15118,-0.616099,-0.205998,1.005295,0.655997,1.076523,-1.040755,1.220398,0.045366,-1.135058,-0.294706,...,1.230245,1.603914,1.539976,0.687581,1.209441,0.899950,1.385534,1.240355,0.359565,1.237662
15119,-0.628070,0.093779,1.360172,-0.043775,0.815245,-1.110942,0.533231,0.790940,-0.350637,-0.526545,...,1.111804,1.232286,1.164963,0.103492,1.276191,0.629594,0.535025,1.029124,0.534831,0.541415
15120,-0.656801,0.366302,2.069925,0.437021,0.439658,-1.089810,-0.775657,1.097941,0.629891,-0.542910,...,1.174169,1.337659,1.258749,0.570232,1.272429,0.788908,0.890299,0.790371,0.654927,0.857800


,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
15115,15116,-0.340758,0.784172,0.768711,0.146639,-0.719765,-0.795476,-1.397379,1.404943,1.719366,...,0,0,0,0,0,0,0,0,0,3
15116,15117,-0.350335,-0.324091,0.295543,1.931773,2.350255,-0.827173,1.187676,0.089223,-0.960742,...,0,0,0,0,0,0,0,0,0,3
15117,15118,-0.616099,-0.205998,1.005295,0.655997,1.076523,-1.040755,1.220398,0.045366,-1.135058,...,0,0,0,0,0,0,0,0,0,3
15118,15119,-0.628070,0.093779,1.360172,-0.043775,0.815245,-1.110942,0.533231,0.790940,-0.350637,...,0,0,0,0,0,0,0,0,0,3
15119,15120,-0.656801,0.366302,2.069925,0.437021,0.439658,-1.089810,-0.775657,1.097941,0.629891,...,0,0,0,0,0,0,0,0,0,3


In [32]:
display(X_test.tail())
display(test_df.tail())

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,"SQUARED_ROOT(Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Hillshade_9am, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Hillshade_3pm, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Aspect, Horizontal_Distance_To_Hydrology)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Hydrology)","SQUARED_ROOT(Hillshade_9am, Horizontal_Distance_To_Hydrology)","SQUARED_ROOT(Elevation, Vertical_Distance_To_Hydrology)","SQUARED_ROOT(Aspect, Horizontal_Distance_To_Fire_Points)","SQUARED_ROOT(Aspect, Hillshade_9am)"
Id,,,,,,,,,,,,,,,,,,,,,
581008,-0.845949,-0.033399,0.413835,-0.676902,-0.556466,-1.212072,0.893176,0.790940,-0.372426,-0.612917,...,1.388278,1.505617,1.267999,0.677726,1.478089,1.083432,1.120696,1.012563,0.613826,0.893800
581009,-0.857920,-0.042483,0.295543,-0.762588,-0.638115,-1.221884,0.893176,0.790940,-0.350637,-0.605643,...,1.440327,1.513527,1.271198,0.763771,1.492992,1.147853,1.174438,1.069214,0.607132,0.894186
581010,-0.869891,0.021106,0.058958,-0.795911,-0.719765,-1.225657,0.762287,0.966369,-0.110952,-0.597461,...,1.461407,1.443370,1.230669,0.796191,1.502979,1.179061,1.102069,1.129058,0.597834,0.762579
581011,-0.874680,0.121031,-0.177626,-0.795911,-0.752425,-1.225657,0.565954,1.141799,0.172311,-0.588369,...,1.461407,1.350014,1.237710,0.805061,1.505756,1.182598,0.976615,1.153780,0.600689,0.578750
581012,-0.877074,0.075610,-0.414210,-0.795911,-0.768755,-1.243015,0.598676,1.097941,0.128732,-0.578368,...,1.475995,1.379674,1.249664,0.799494,1.521297,1.184370,0.995935,1.166294,0.583290,0.603432


,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
565887,581008,-0.845949,-0.033399,0.413835,-0.676902,-0.556466,-1.212072,0.893176,0.790940,-0.372426,...,0,0,0,0,0,0,0,0,0,NaN
565888,581009,-0.857920,-0.042483,0.295543,-0.762588,-0.638115,-1.221884,0.893176,0.790940,-0.350637,...,0,0,0,0,0,0,0,0,0,NaN
565889,581010,-0.869891,0.021106,0.058958,-0.795911,-0.719765,-1.225657,0.762287,0.966369,-0.110952,...,0,0,0,0,0,0,0,0,0,NaN
565890,581011,-0.874680,0.121031,-0.177626,-0.795911,-0.752425,-1.225657,0.565954,1.141799,0.172311,...,0,0,0,0,0,0,0,0,0,NaN
565891,581012,-0.877074,0.075610,-0.414210,-0.795911,-0.768755,-1.243015,0.598676,1.097941,0.128732,...,0,0,0,0,0,0,0,0,0,NaN


# Training

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.metrics import accuracy_score

from tqdm import tqdm_notebook

from collections import OrderedDict

def fit_predict(model, X_train=X_train, y_train=y_train, 
                        X_test=X_test, y_test=y_test):
    
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    score = accuracy_score(predictions, y_test)
    
    return score


def fit_pipelines(pipelines, 
                  X_train=X_train, y_train=y_train,
                 X_test=X_test, y_test=y_test):
    
    scores = {}
    for name, pipeline in tqdm_notebook(pipelines.items()):
        print('Start fitting using {}'.format(name))
        pipeline.fit(X_train, y_train)

        predictions = pipeline.predict(X_test)
        score = accuracy_score(predictions, y_test)
        scores[name] = score

        print('{:20s}: score={}'.format(name, score))

    return(scores)

In [34]:
%%time


seed=47
pipelines = {}

pipelines['Random Forest'] = Pipeline(steps=[('RF', 
                                            RandomForestClassifier(n_jobs=-1,
                                                                   n_estimators=300,
#                                                                    max_features=7,
#                                                                    max_leaf_nodes=220,
                                                                   random_state=seed
                                                                   )
                                                    ),])

pipelines['Extra Trees'] = Pipeline(
                                    steps=[('random forest extra', ExtraTreesClassifier(n_estimators=100, random_state=seed))]
                                    )

# pipelines['KNN'] = Pipeline(steps=[
#                                         ('KNN', KNeighborsClassifier(n_neighbors=2))
#                                     ])

pipelines['LGBM'] = Pipeline(steps=[('LightGBM', 
                                    lgb.LGBMClassifier(n_estimators=300,
                                                             num_class=8,
                                                            num_leaves=32,
                                                            random_state=seed,)
                                   ),])



scores = fit_pipelines(pipelines)

Start fitting using Random Forest
Random Forest       : score=0.731888770295392
Start fitting using Extra Trees
Extra Trees         : score=0.7684045718971111
Start fitting using LGBM
LGBM                : score=0.7588444438161345

CPU times: user 7min 47s, sys: 7.79 s, total: 7min 54s
Wall time: 2min 17s


# Feature Selection

In [35]:
from sklearn.feature_selection import SelectFromModel


In [36]:
features_var = pd.DataFrame(index=X_train.columns)
name_classifiers = []

for name, p in tqdm_notebook(pipelines.items()):
    
    # cannot apply feature selection to KNN
    if 'KNN'.lower() in name.lower():
        continue

    name_classifiers.append(name)
    
#     step_name = p.steps[-1][0]
#     print(step_name)
    clf = p.steps[-1][1]
#     print(clf)

    model = SelectFromModel(clf, prefit=True)
    X_train_new = model.transform(X_train)
    X_train_inversed = model.inverse_transform(X_train_new)
    selected_features = pd.DataFrame(X_train_inversed,
                                    index=X_train.index,
                                    columns=X_train.columns)
    
    features_var[name] = selected_features.var()
    

In [37]:
features_var['mean'] = features_var[name_classifiers].mean(axis=1)
features_var['var'] = features_var[name_classifiers].var(axis=1)

features_var[:20]

,Random Forest,Extra Trees,LGBM,mean,var
Elevation,1.000066,1.000066,1.000066,1.000066,0.000000
Aspect,0.000000,0.000000,0.000000,0.000000,0.000000
Slope,0.000000,0.000000,0.000000,0.000000,0.000000
Horizontal_Distance_To_Hydrology,0.000000,1.000066,0.000000,0.333355,0.333377
Vertical_Distance_To_Hydrology,0.000000,0.000000,0.000000,0.000000,0.000000
Horizontal_Distance_To_Roadways,1.000066,1.000066,1.000066,1.000066,0.000000
Hillshade_9am,0.000000,0.000000,0.000000,0.000000,0.000000
Hillshade_Noon,0.000000,0.000000,0.000000,0.000000,0.000000
Hillshade_3pm,0.000000,0.000000,0.000000,0.000000,0.000000
Horizontal_Distance_To_Fire_Points,1.000066,1.000066,1.000066,1.000066,0.000000


# Mean/Variance

### Mean

In [38]:
display(pd.DataFrame(features_var['mean'].sort_values(ascending=False)))


,mean
Elevation + Horizontal_Distance_To_Roadways,3.157527
Horizontal_Distance_To_Fire_Points + Horizontal_Distance_To_Roadways,2.972968
Elevation + Horizontal_Distance_To_Fire_Points,2.887318
Elevation + Hillshade_Noon,2.431725
Aspect - Elevation,2.022326
...,...
"MEAN_3_COLS(Aspect, Elevation, Hillshade_3pm)",0.000000
"MEAN_2_COLS(Aspect, Hillshade_9am)",0.000000
Hillshade_3pm + Slope,0.000000
"MEAN_2_COLS(Hillshade_9am, Horizontal_Distance_To_Hydrology)",0.000000


### Variance

In [39]:
n_top = 10
display(pd.DataFrame(features_var['var'].sort_values(ascending=False)))

,var
Horizontal_Distance_To_Hydrology + Vertical_Distance_To_Hydrology,3.639914
Horizontal_Distance_To_Hydrology * Vertical_Distance_To_Hydrology,3.241464
Elevation + Horizontal_Distance_To_Hydrology,2.661357
Elevation - Slope,2.297667
Horizontal_Distance_To_Fire_Points - Slope,2.048841
...,...
Elevation * Hillshade_Noon,0.000000
Hillshade_3pm * Slope,0.000000
Elevation * Horizontal_Distance_To_Fire_Points,0.000000
Elevation * Hillshade_3pm,0.000000


### Show Top-10 mean and variation

In [40]:
# num_selected_features = 50
# num_selected_features = 22  #Extra Trees         : score= 0.7791504385995914
num_selected_features = 16 # Extra Trees         : score= 0.7796929449435581
# num_selected_features = 15 # Extra Trees         : score=0.7723699928608285
# num_selected_features = 14 # Extra Trees         : score= 0.7738101969987206
# num_selected_features = 5 # Extra Trees         : score=0.7738296353367781


original_features = list(data_df.columns)
original_features.remove('Id')
original_features = set(original_features)

engineered_features = set(X_train) - original_features

top_mean = features_var['mean'].sort_values(ascending=False)[:num_selected_features]
top_var = features_var['var'].sort_values(ascending=False)[:num_selected_features]

top_mean_features = set(top_mean.index).intersection(engineered_features)
top_var_features = set(top_var.index).intersection(engineered_features)

print('original_features: {}'.format(len(original_features)))
print('top_mean_features: {}'.format(len(top_mean_features)))
print('top_var_features: {}'.format(len(top_var_features)))

print('top_mean_features + top_var_features: {}'.format(len(
                                                            top_mean_features.union(top_var_features)
                                                           
                                                           )))


selected_top_features = original_features.union(
                                                top_mean_features.union(top_var_features)
                                               )

print('Total Features: {}'.format(len(selected_top_features)))
# print()
# print(top_mean_features)
# print()
# print(top_var_features)
# print()
# print(top_mean_features.union(top_var_features))
# print()

original_features: 54
top_mean_features: 16
top_var_features: 16
top_mean_features + top_var_features: 28
Total Features: 82


### Top-10 Mean

In [41]:
pd.options.display.max_colwidth = 100

display(pd.DataFrame(features_var.loc[top_mean_features, 'mean'].sort_values(ascending=False)))


,mean
Elevation + Horizontal_Distance_To_Roadways,3.157527
Horizontal_Distance_To_Fire_Points + Horizontal_Distance_To_Roadways,2.972968
Elevation + Horizontal_Distance_To_Fire_Points,2.887318
Elevation + Hillshade_Noon,2.431725
Aspect - Elevation,2.022326
Horizontal_Distance_To_Fire_Points * Horizontal_Distance_To_Roadways,1.910398
Elevation + Horizontal_Distance_To_Hydrology,1.883740
Elevation - Slope,1.750302
Elevation - Hillshade_Noon,1.568540
Elevation + Vertical_Distance_To_Hydrology,1.496221


### Top-10 Variance

In [42]:
display(pd.DataFrame(features_var.loc[top_var_features, 'var'].sort_values(ascending=False)))

,var
Horizontal_Distance_To_Hydrology + Vertical_Distance_To_Hydrology,3.639914
Horizontal_Distance_To_Hydrology * Vertical_Distance_To_Hydrology,3.241464
Elevation + Horizontal_Distance_To_Hydrology,2.661357
Elevation - Slope,2.297667
Horizontal_Distance_To_Fire_Points - Slope,2.048841
Horizontal_Distance_To_Hydrology + Horizontal_Distance_To_Roadways,1.931141
Hillshade_Noon + Horizontal_Distance_To_Fire_Points,1.679027
Elevation + Vertical_Distance_To_Hydrology,1.679008
Hillshade_9am - Vertical_Distance_To_Hydrology,1.601628
Hillshade_3pm + Horizontal_Distance_To_Hydrology,1.557805


# Re-Train using selected features

In [43]:
fit_pipelines(pipelines, 
                X_train=X_train[selected_top_features], y_train=y_train,
                 X_test=X_test[selected_top_features], y_test=y_test)

Start fitting using Random Forest
Random Forest       : score=0.7454673329893337
Start fitting using Extra Trees
Extra Trees         : score=0.7802725608419981
Start fitting using LGBM
LGBM                : score=0.7614668523322471



{'Random Forest': 0.7454673329893337,
 'Extra Trees': 0.7802725608419981,
 'LGBM': 0.7614668523322471}

# Save Features

In [44]:
feature_objs = []

all_selected_features = list(top_mean_features.union(top_var_features))
for f in features:
    if f.get_name() in all_selected_features:
        feature_objs.append(f)

In [45]:
len(all_selected_features)

28

In [46]:
ft.save_features(feature_objs,
                location='./features.json')

### Calculating Feature Matrix for New Data

In [47]:
saved_features = ft.load_features('./features.json')

#### calculate the feature matrix.

In [48]:
%%time
feature_matrix_restored = ft.calculate_feature_matrix(saved_features, es)

CPU times: user 6.8 s, sys: 864 ms, total: 7.66 s
Wall time: 6.86 s


In [49]:
feature_matrix_restored.shape

(581012, 28)

In [50]:
feature_matrix_restored.head()

,Hillshade_Noon + Horizontal_Distance_To_Fire_Points,Hillshade_3pm + Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Hydrology + Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Fire_Points + Horizontal_Distance_To_Roadways,Elevation + Hillshade_Noon,Elevation + Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Fire_Points + Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Hydrology + Horizontal_Distance_To_Roadways,Elevation + Horizontal_Distance_To_Roadways,Elevation + Horizontal_Distance_To_Hydrology,...,Elevation - Slope,Elevation - Hillshade_3pm,Hillshade_9am - Horizontal_Distance_To_Roadways,Hillshade_9am - Horizontal_Distance_To_Hydrology,Aspect - Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Hydrology * Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Fire_Points * Horizontal_Distance_To_Roadways,Elevation * Hillshade_9am,Elevation * Horizontal_Distance_To_Hydrology,Elevation * Vertical_Distance_To_Hydrology
Id,,,,,,,,,,,,,,,,,,,,,
1,4.906459,0.427898,-0.687435,3.426124,0.204558,3.967710,3.500731,-0.762042,-1.275776,-0.220456,...,1.230037,-0.648354,1.180135,0.124814,-5.294785,-0.122308,-3.938956,-0.099649,-0.053831,0.306185
2,4.988935,0.274290,-1.004391,3.286464,0.321765,3.904249,3.353656,-1.071583,-1.380707,-0.453798,...,1.333963,-0.728088,1.237977,0.311068,-5.200269,0.067422,-4.282478,-0.091067,0.027594,0.355542
3,5.025953,0.192238,0.421612,5.297535,0.965710,4.322068,4.418525,1.300622,1.237291,0.325155,...,1.018291,0.132917,-0.409536,0.502600,-4.351733,0.044165,4.637006,0.091225,0.025429,0.029765
4,5.107779,-0.214794,1.163327,5.311437,0.920219,4.358403,5.365835,1.108929,1.123877,0.155895,...,-0.091829,0.370689,-0.210724,0.757258,-4.288212,0.077017,4.437300,0.070706,0.006020,0.093353
5,4.896892,-0.028360,-1.203602,3.239033,0.289879,3.868034,3.387119,-1.351689,-1.367980,-0.722687,...,1.345934,-0.694327,1.237223,0.591929,-5.252008,0.300361,-4.231130,-0.088209,0.130503,0.314215


# Stacking

In [51]:
# classifiers = []

# for name, p in pipelines.items():
#     if 'KNN'.lower() in name.lower():
#         continue
        
#     classifiers.append(p.steps[-1][1])

In [52]:
# %%time
# from mlxtend.classifier import EnsembleVoteClassifier

# eclf = EnsembleVoteClassifier(clfs=classifiers,
#                              voting='soft')


# score = fit_predict(eclf)

# print('score={}'.format(score))


In [53]:
# %%time
# from mlxtend.classifier import EnsembleVoteClassifier

# eclf = EnsembleVoteClassifier(clfs=classifiers,
#                              voting='hard')


# score = fit_predict(eclf)

# print('score={}'.format(score))
